## Goals

- Open a roman science file with the python asdf library
- Use tools for looking at the file and searching for attributes
- Display an image contained in the file
- Access and modify metadata
- Write the file to disk

The file roman.asdf is presumed to be in the tutorial's data/ directory. This is simulated data from the [OpenUniverse2024 project](https://irsa.ipac.caltech.edu/data/theory/openuniverse2024/overview.html) packaged as a level 2 calibrated data model

In [ ]:
import numpy as np
import asdf

from matplotlib import pyplot as plt
%matplotlib widget

Load ASDF file
--------------

In [ ]:
af = asdf.open('../data/roman.asdf')
af

## Info
The file can be rendered using the **info** method

In [ ]:
af.info()

In [ ]:
# The file has lots of content so some options are needed to see it all

af.info(max_rows=200)

**info** is also a utility function. If called like that it can render specific nodes in the tree.

And there's a command line utility **asdftool info** which allows looking at files outside a Python session.

In [ ]:
!asdftool info ../data/roman.asdf

## Search
Another useful utility is **search** - search by key, value or type.

In [ ]:
# Search for attributes with exposure in their name/key
af.search('exposure')

Regular expressions are supported (and can be used to find exact matches).

In [ ]:
af.search('^exposure_time$')

Search returns an AsdfSearchResult, use `node` to get the value at that node.

In [ ]:
result = af.search('^exposure_time$')
type(result)

In [ ]:
result.node

## Dictionary-like access

AsdfFile instances can also be used much like a python dictionary where values are accessed by key.

In [ ]:
# Accessing attributes: assign an attribute value to a variable
exposure_time = af['roman']['meta']['exposure']['exposure_time']
exposure_time

## Tree content

Although an ASDF file can contain both a human readable YAML header and binary blocks the entire contents of the file is accessable via the "tree". Put another way, array and table data is not separate and can be nested within the "tree". Let's look at the "data" array in this file.

In [ ]:
# Access the "data" under the "roman" sub-tree
im = af['roman']['data']
# Plot the data (with restricted values selected for this file)
fig, ax = plt.subplots()
ax.imshow(im, vmin=18, vmax=200, origin='lower')

In [ ]:
# let's apply a simple binary threshold to the image
thresholded_image = im > (im.mean() + im.std() * 3)
fig1, ax1 = plt.subplots()
ax1.imshow(thresholded_image, cmap="gray", origin='lower')

Modifying the contents
----------------------

In [ ]:
# Arbitrarily change the exposure_time
af['roman']['meta']['exposure']['exposure_time'] = 1800
af['roman']['meta']['exposure']['exposure_time']

In [ ]:
# Replace data with a log version of the image.
af['roman']['data'] = np.log(af['roman']['data'])

In [ ]:
# Create new meta attribute
af['inspected_by'] = 'Jacques Clouseau'

Saving the modified contents
----------------------------

In [ ]:
# Save to a different file
af.write_to('roman_modified.asdf')

In [ ]:
ls

## Roman datamodels
Roman ASDF files can also be opened with [roman_datamodels](https://roman-datamodels.readthedocs.io/en/latest/) which provides a slightly different interface and is used by the [romancal](https://roman-pipeline.readthedocs.io/en/latest/) calibration pipeline.

In [ ]:
import roman_datamodels
model = roman_datamodels.open("../data/roman.asdf")
print(model)

The major differences with the datamodel and "vanilla" asdf interface are:
- there is no top level "roman" key/attribute
- items are accessed as attributes (`model.meta`) instead of by key (`af['roman']['meta']`)
- saving uses [save](https://roman-datamodels.readthedocs.io/en/latest/api/roman_datamodels.datamodels.DataModel.html#roman_datamodels.datamodels.DataModel.save) instead of [write_to](https://asdf.readthedocs.io/en/latest/api/asdf.AsdfFile.html#asdf.AsdfFile.write_to)

In [ ]:
model.meta.exposure.exposure_time

In [ ]:
model.save("new_file.asdf")

Exercises
---------

1. Use `search` to find what detector was used for this simulated image.
2. Where is the "detector" key located within the ASDF tree?
3. Access the "detector" key using the `AsdfFile` interface (opening the file with `asdf.open`).
4. Access the "detector" attribute using the `DataModel` interface (opening the file with `roman_datamodels.open`).